In [21]:
import pandas as pd
import numpy as np
import networkx as nx # 引入网络分析库

![jupyter](参考文献/AMJ.png)

In [22]:
# df = pd.read_excel('上市公司社会责任评价指标表/CSRR_EvaluationIndex.xlsx')
# df.to_csv('上市公司社会责任评价指标表/CSRR_EvaluationIndex.csv',index=False,encoding='utf-8-sig')

## 加载企业的CSR议题披露数据

In [23]:
file_path = '上市公司社会责任评价指标表/CSRR_EvaluationIndex.csv'
df = pd.read_csv(file_path,skiprows=[0,2],parse_dates=['统计截止日期'],usecols=['股票代码','统计截止日期','披露内容类型编码'])
df['Year'] = df['统计截止日期'].dt.year
del df['统计截止日期']
# 1.1 重命名列以方便代码中使用
df.rename(columns={
    '股票代码': 'StockCode',
    '年份': 'Year',
    '披露内容类型编码': 'DisclosureType'
}, inplace=True)
# 1.2 数据类型转换
# df['StockCode'] = df['StockCode'].astype(str)
df['Year'] = df['Year'].astype(int)
df['DisclosureType'] = df['DisclosureType'].astype(str)
print(f'数据共{df.shape[0]}行')
df

数据共332076行


,StockCode,DisclosureType,Year
0,1,S3303,2008
1,1,S3303,2009
2,1,S3303,2009
3,1,S3303,2010
4,1,S3303,2010
...,...,...,...
332071,920985,S3303,2024
332072,920985,S3303,2024
332073,920985,S3303,2024
332074,920985,S3309,2024


## 加载企业的行业代码（SIC）数据

In [24]:
# 打开文件并读取内容
with open("公司基本情况文件/CG_Co[DES][xlsx].txt", "r", encoding="utf-8") as file:
    content = file.read()
print(content)

Stkcd [证券代码] - 以交易所公布的证券代码为准。
Stknme [证券中文简称] - 以交易所公布的中文简称为准
ListedDate [上市日期] - 
Stktype [股票类型] - A=A股，B=B股，CDR=中国存托凭证
Crcd [A/B/H股交叉码] - 指该公司发行的其他证券代码，以交易所公布的证券代码为准。
Conme [公司名称] - 以交易所公布的中文全称为准。
Cochsnm [公司中文简称] - 以交易所公布的信息为准。
Conmee [公司英文名称] - 以交易所公布的信息为准。
IndustryNameD [行业名称D] - 中国上市公司协会行业分类名称。
IndustryCodeD [行业代码D] - 中国上市公司协会行业分类代码。
Nnindnme [行业名称C] - 2012版证监会行业分类名称
Nnindcd [行业代码C] - 2012版证监会行业分类代码
Nindnme [行业名称B] - 中国证监会( CSRC)(2001年版) 《上市公司行业分类指引》制定。
Nindcd [行业代码B] - 中国证监会( CSRC)(2001年版) 《上市公司行业分类指引》制定。
Indnme [行业名称A] - 中国证监会( CSRC)(1999年版) 《上市公司行业分类指引》制定
Indcd [行业代码A] - 中国证监会( CSRC)(1999年版) 《上市公司行业分类指引》制定
Busscope [经营范围] - 经营范围及主要产品或提供劳务
Cohisty [公司沿革] - 公司历史沿革、改制简况（自2006年始，删除其中经营范围、主要产品及提供劳务相关内容）
Regcap [注册资本] - 以报告中公司简介信息为准
EstablishDate [成立日期] - 
DelistedDate [退市日期] - 


In [25]:
industry_code = pd.read_excel('公司基本情况文件/CG_Co.xlsx',usecols=['Stkcd','Nnindcd','Stktype'],skiprows=[1,2])
industry_code.rename(columns={'Stkcd':'StockCode','Nnindcd':'IndustryCode'},inplace=True) #选择2012版证监会行业分类代码
# industry_code['StockCode'] = industry_code['StockCode'].astype(str)
# industry_code = industry_code[industry_code['Stktype'] == 'A']
industry_code

d:\Anaconda\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,StockCode,Stktype,IndustryCode
0,1,A,J66
1,2,A,K70
2,3,A,S90
3,4,A,I65
4,5,A,N77
...,...,...,...
5889,920978,A,NaN
5890,920981,A,C39
5891,920982,A,NaN
5892,920985,A,C38


In [26]:
industry_code.isnull().sum()

StockCode         0
Stktype           0
IndustryCode    414
dtype: int64

In [27]:
df = pd.merge(df,industry_code,on='StockCode',how='left') # 合并行业代码
df

,StockCode,DisclosureType,Year,Stktype,IndustryCode
0,1,S3303,2008,A,J66
1,1,S3303,2009,A,J66
2,1,S3303,2009,A,J66
3,1,S3303,2010,A,J66
4,1,S3303,2010,A,J66
...,...,...,...,...,...
332071,920985,S3303,2024,A,C38
332072,920985,S3303,2024,A,C38
332073,920985,S3303,2024,A,C38
332074,920985,S3309,2024,A,C38


In [28]:
df.isnull().sum() # 检查缺失值

StockCode            0
DisclosureType       0
Year                 0
Stktype              0
IndustryCode      5023
dtype: int64

In [29]:
df = df[df['IndustryCode'].notna()] # 移除缺失行业代码的行
df

,StockCode,DisclosureType,Year,Stktype,IndustryCode
0,1,S3303,2008,A,J66
1,1,S3303,2009,A,J66
2,1,S3303,2009,A,J66
3,1,S3303,2010,A,J66
4,1,S3303,2010,A,J66
...,...,...,...,...,...
332071,920985,S3303,2024,A,C38
332072,920985,S3303,2024,A,C38
332073,920985,S3303,2024,A,C38
332074,920985,S3309,2024,A,C38


In [30]:
# 1.3 关键参数
K = 9  # CSR议题总数

# 1.4 计算每个公司在某个年份的某个CSR议项上的原始总披露次数 (x_ikt)。每行代表一次披露，因此计数即为投入量 X_ikt。
disclosure_counts = df.groupby(['StockCode', 'Year', 'DisclosureType', 'IndustryCode']).size().reset_index(name='X_ikt')
disclosure_counts

,StockCode,Year,DisclosureType,IndustryCode,X_ikt
0,1,2008,S3303,J66,1
1,1,2009,S3303,J66,2
2,1,2010,S3302,J66,1
3,1,2010,S3303,J66,3
4,1,2010,S3305,J66,1
...,...,...,...,...,...
68672,920985,2024,S3303,C38,23
68673,920985,2024,S3305,C38,6
68674,920985,2024,S3306,C38,9
68675,920985,2024,S3307,C38,1


In [31]:
# 1.5 处理缺失值：创建完整的 Company-Year-Type 组合并填充 0
all_companies = disclosure_counts['StockCode'].unique()
all_years = disclosure_counts['Year'].unique()
all_disclosure_types = disclosure_counts['DisclosureType'].unique()

# 检查议题数量是否匹配 (可选)
if len(all_disclosure_types) != K:
    print(f"警告：检测到 {len(all_disclosure_types)} 个唯一议题，与您提供的 K={K} 不一致。代码将使用检测到的数量。")
else:
    print(f"检测到 {len(all_disclosure_types)} 个唯一议题，与您提供的 K={K} 一致。")

检测到 9 个唯一议题，与您提供的 K=9 一致。


- `pd.MultiIndex.from_product(...)`： 这一行执行了这三个列表（公司、年份、议题）的笛卡尔积 (Cartesian Product)。
- 如果：公司有 $N$ 个，年份有 $T$ 个，议题有 $K$ 个（$K=9$）。那么，`multi_index` 会创建 $N \times T \times K$ 个组合，代表了“理论上”所有公司在所有年份对所有议题的投入记录。
- `full_df`： 这是一个全新的、完整的数据框架，包含了所有可能的 StockCode-Year-DisclosureType 组合。

In [32]:
# 创建一个包含所有 Company-Year-Type 组合的 DataFrame
multi_index = pd.MultiIndex.from_product([all_companies, all_years, all_disclosure_types], names=['StockCode', 'Year', 'DisclosureType'])
full_df = pd.DataFrame(index=multi_index).reset_index()
full_df

,StockCode,Year,DisclosureType
0,1,2008,S3303
1,1,2008,S3302
2,1,2008,S3305
3,1,2008,S3306
4,1,2008,S3309
...,...,...,...
458275,920985,2006,S3309
458276,920985,2006,S3308
458277,920985,2006,S3301
458278,920985,2006,S3304


In [33]:
# 找出 StockCode-Year 对应的 IndustryCode 映射
# company_industry_map = disclosure_counts[['StockCode', 'Year', 'IndustryCode']].drop_duplicates()
# company_industry_map

In [34]:
# 将实际的披露次数 (X_ikt) 合并到完整框架中，未匹配项填充 0
X_ikt_df = full_df.merge(disclosure_counts[['StockCode', 'Year', 'DisclosureType', 'X_ikt']], 
                       on=['StockCode', 'Year', 'DisclosureType'], 
                       how='left').fillna({'X_ikt': 0})
X_ikt_df

,StockCode,Year,DisclosureType,X_ikt
0,1,2008,S3303,1.0
1,1,2008,S3302,0.0
2,1,2008,S3305,0.0
3,1,2008,S3306,0.0
4,1,2008,S3309,0.0
...,...,...,...,...
458275,920985,2006,S3309,0.0
458276,920985,2006,S3308,0.0
458277,920985,2006,S3301,0.0
458278,920985,2006,S3304,0.0


In [35]:
# 重新添加 IndustryCode
X_ikt_df = pd.merge(X_ikt_df,industry_code, on=['StockCode'], how='left')
print(X_ikt_df.isnull().sum())
X_ikt_df

StockCode         0
Year              0
DisclosureType    0
X_ikt             0
Stktype           0
IndustryCode      0
dtype: int64


,StockCode,Year,DisclosureType,X_ikt,Stktype,IndustryCode
0,1,2008,S3303,1.0,A,J66
1,1,2008,S3302,0.0,A,J66
2,1,2008,S3305,0.0,A,J66
3,1,2008,S3306,0.0,A,J66
4,1,2008,S3309,0.0,A,J66
...,...,...,...,...,...,...
458275,920985,2006,S3309,0.0,A,C38
458276,920985,2006,S3308,0.0,A,C38
458277,920985,2006,S3301,0.0,A,C38
458278,920985,2006,S3304,0.0,A,C38


## 计算行业平均投入$(\bar{x}_{jkt})$ 
$$\bar{x}_{jkt} = \frac{1}{N_{jt}} \sum_{i \in j} x_{ikt}$$

In [36]:
# 行业-年份-议题 的平均投入 Mean_X_jkt
industry_mean_X_jkt = X_ikt_df.groupby(['IndustryCode', 'Year', 'DisclosureType'])['X_ikt'].mean().reset_index(name='Mean_X_jkt')

# 合并回主数据框
X_ikt_df = X_ikt_df.merge(industry_mean_X_jkt, on=['IndustryCode', 'Year', 'DisclosureType'], how='left')
X_ikt_df


,StockCode,Year,DisclosureType,X_ikt,Stktype,IndustryCode,Mean_X_jkt
0,1,2008,S3303,1.0,A,J66,0.069767
1,1,2008,S3302,0.0,A,J66,0.023256
2,1,2008,S3305,0.0,A,J66,0.000000
3,1,2008,S3306,0.0,A,J66,0.023256
4,1,2008,S3309,0.0,A,J66,0.000000
...,...,...,...,...,...,...,...
458275,920985,2006,S3309,0.0,A,C38,0.000000
458276,920985,2006,S3308,0.0,A,C38,0.000000
458277,920985,2006,S3301,0.0,A,C38,0.000000
458278,920985,2006,S3304,0.0,A,C38,0.000000


## 计算Eigenvector Centrality (CEN)

1. 双模从属矩阵 (`M`)，Firms × Issues，创建一个矩阵，行是公司，列是 9 个议题，值是 `xikt`​（披露次数）。
2. 单模概念网络 (`W`)，Issues × Issues，将 M 转换成一个 议题-议题 的共现矩阵，表示两个议题共同被公司披露的紧密程度。`W=MTM`
3. 特征向量中心性 (`CEN`)，Issue Importance，"计算 W 矩阵中每个议题的特征向量中心性。这个值 (`CEN_k,t`​) 即为该议题在该行业-年份中的相对重要性权重。"

In [37]:
# 初始化一个列表来存储计算出的 CEN 结果
cen_results = []

# 对每个行业-年份分组进行迭代计算
grouped_data = X_ikt_df.groupby(['IndustryCode', 'Year'])

for name, group in grouped_data:
    industry_code, year = name
    
    # 1. 创建 双模从属矩阵 (Firms x Issues)
    # Pivot table to get the Affiliation Matrix (M) for this Industry-Year
    # S_ikt 只需要披露与否，所以先二元化
    group['S_ikt'] = np.where(group['X_ikt'] > 0, 1, 0)
    
    # 使用 S_ikt 而非 X_ikt 作为值，因为论文公式使用 DUMMIES for each issue field
    affiliation_matrix_S = group.pivot_table(
        index='StockCode', 
        columns='DisclosureType', 
        values='S_ikt', 
        fill_value=0 # 确保未披露的矩阵值为 0
    )
    
    # 2. 转换成 单模共现矩阵 (Issues x Issues)
    # W = M^T M
    W = affiliation_matrix_S.T.dot(affiliation_matrix_S)
    
    # 3. 计算 Eigenvector Centrality (CEN)
    # 转换为 networkx 图对象
    G = nx.Graph(W.values)
    
    try:
        # 计算特征向量中心性
        # CEN 结果是一个字典，键为节点索引 (0到K-1)，值为中心性分数
        cen_scores = nx.eigenvector_centrality(G, max_iter=1000) 
        
        # 将结果转换为 DataFrame
        cen_df = pd.DataFrame(list(cen_scores.items()), columns=['DisclosureType_Index', 'CEN'])
        cen_df['DisclosureType'] = affiliation_matrix_S.columns.values[cen_df['DisclosureType_Index']]
        
        # 标准化 CEN 分数，使其更容易解释 (可选，但推荐)
        total_cen = cen_df['CEN'].sum()
        cen_df['CEN'] = cen_df['CEN'] / total_cen
        
        cen_df['IndustryCode'] = industry_code
        cen_df['Year'] = year
        
        cen_results.append(cen_df[['IndustryCode', 'Year', 'DisclosureType', 'CEN']])
        
    except nx.exception.PowerIterationFailedConvergence:
        # 如果计算不收敛，通常发生在网络太稀疏或太小的情况下。
        # 对于这些情况，将 CEN 设为 0 或 NaN，表示无法确定权重。
        print(f"警告: {industry_code}, {year} 行业/年份网络收敛失败，CEN设为0。")
        cen_df_failed = pd.DataFrame({
            'DisclosureType': affiliation_matrix_S.columns.values,
            'CEN': 0.0,
            'IndustryCode': industry_code,
            'Year': year
        })
        cen_results.append(cen_df_failed)
        print(cen_results)


# 合并所有 CEN 结果
cen_df_final = pd.concat(cen_results, ignore_index=True)

# 4. 滞后 CEN 权重 (CEN_k, t-1)
cen_df_final['Year_Lag'] = cen_df_final['Year'] + 1 
cen_df_final.rename(columns={'CEN': 'CEN_t_minus_1'}, inplace=True)

# 5. 合并 CEN_t_minus_1 到主数据框
X_ikt_df = X_ikt_df.merge(
    cen_df_final[['IndustryCode', 'Year_Lag', 'DisclosureType', 'CEN_t_minus_1']],
    left_on=['IndustryCode', 'Year', 'DisclosureType'],
    right_on=['IndustryCode', 'Year_Lag', 'DisclosureType'],
    how='left'
).drop(columns=['Year_Lag'])

# 填充第一年的 CEN 缺失值（因为没有 t-1 的数据，通常填 0 或行业平均，这里填 0）
X_ikt_df['CEN_t_minus_1'] = X_ikt_df['CEN_t_minus_1'].fillna(0)
X_ikt_df

,StockCode,Year,DisclosureType,X_ikt,Stktype,IndustryCode,Mean_X_jkt,CEN_t_minus_1
0,1,2008,S3303,1.0,A,J66,0.069767,4.999989e-01
1,1,2008,S3302,0.0,A,J66,0.023256,4.999989e-01
2,1,2008,S3305,0.0,A,J66,0.000000,3.136120e-07
3,1,2008,S3306,0.0,A,J66,0.023256,3.136120e-07
4,1,2008,S3309,0.0,A,J66,0.000000,3.136120e-07
...,...,...,...,...,...,...,...,...
458275,920985,2006,S3309,0.0,A,C38,0.000000,0.000000e+00
458276,920985,2006,S3308,0.0,A,C38,0.000000,0.000000e+00
458277,920985,2006,S3301,0.0,A,C38,0.000000,0.000000e+00
458278,920985,2006,S3304,0.0,A,C38,0.000000,0.000000e+00


## 计算 Scope Conformity (范围一致性)
指标定义: 
### $$\text{Scope Conformity}_{it} = \sum_{k=1}^9 \text{CSR}_{ikt} \cdot \text{CEN}_{k, t-1}$$
- $CSR_{ikt} = 1$ if $x_{ikt} > 0$, else $0$.

In [38]:
# 1. 确保 S_ikt 已经计算 (公司 i 在 t 年是否披露了议题 k)
X_ikt_df['S_ikt'] = np.where(X_ikt_df['X_ikt'] > 0, 1, 0) # 对应论文中的 CSRit

# 2. 计算 Scope Conformity: Sum(CSRit * CEN_k, t-1)
X_ikt_df['ScopeConformity_k'] = X_ikt_df['S_ikt'] * X_ikt_df['CEN_t_minus_1']

# 3. 对所有 k (议题) 求和
scope_conformity_df = X_ikt_df.groupby(['StockCode', 'Year'])['ScopeConformity_k'].sum().reset_index(name='ScopeConformity')
scope_conformity_df

,StockCode,Year,ScopeConformity
0,1,2006,0.000000
1,1,2007,0.000000
2,1,2008,0.499999
3,1,2009,0.250000
4,1,2010,0.675390
...,...,...,...
50915,920985,2020,0.000000
50916,920985,2021,0.000000
50917,920985,2022,0.000000
50918,920985,2023,0.000000


In [39]:
scope_conformity_df.describe()

,StockCode,Year,ScopeConformity
count,50920.000000,50920.000000,50920.000000
mean,372755.376866,2015.000000,0.171849
std,280554.038736,5.477279,0.280414
min,1.000000,2006.000000,0.000000
25%,2562.500000,2010.000000,0.000000
50%,600069.000000,2015.000000,0.000000
75%,601515.750000,2020.000000,0.332247
max,920985.000000,2024.000000,1.000000


## 计算 Emphasis Differentiation (重点差异化)
指标定义: 
### $$\text{Emphasis Differentiation}_{it} = \sum_{k=1}^K \left( |\text{FE}_{ikt} - \text{AE}_{jkt}| \cdot \mathbf{\text{CEN}_{k, t-1}} \right)$$

In [40]:
# 4.1 计算公司总投入和行业总平均投入 (用于计算比例 P_ikt 和 P_jkt_bar)
company_total_X = X_ikt_df.groupby(['StockCode', 'Year'])['X_ikt'].sum().reset_index(name='Total_X_it')
X_ikt_df = X_ikt_df.merge(company_total_X, on=['StockCode', 'Year'], how='left')

industry_total_mean_X = X_ikt_df.groupby(['IndustryCode', 'Year'])['Mean_X_jkt'].sum().reset_index(name='Total_Mean_X_jt')
X_ikt_df = X_ikt_df.merge(industry_total_mean_X, on=['IndustryCode', 'Year'], how='left')

# 4.2 计算投入比例 (FEit 和 AEjt)
# P_ikt (FEit): Firm's Emphasis
X_ikt_df['P_ikt'] = np.where(X_ikt_df['Total_X_it'] > 0, X_ikt_df['X_ikt'] / X_ikt_df['Total_X_it'], 0) 
# P_jkt_bar (AEjt): Averaged Emphasis
X_ikt_df['P_jkt_bar'] = np.where(X_ikt_df['Total_Mean_X_jt'] > 0, X_ikt_df['Mean_X_jkt'] / X_ikt_df['Total_Mean_X_jt'], 0)

# 4.3 计算加权绝对差求和
# |FEit - AEjt|
X_ikt_df['Absolute_Diff_k'] = np.abs(X_ikt_df['P_ikt'] - X_ikt_df['P_jkt_bar'])

# |FEit - AEjt| * CEN_k, t-1
X_ikt_df['Weighted_Diff_k'] = X_ikt_df['Absolute_Diff_k'] * X_ikt_df['CEN_t_minus_1']

# Sum(|FEit - AEjt| * CEN_k, t-1)
emphasis_diff_df = X_ikt_df.groupby(['StockCode', 'Year'])['Weighted_Diff_k'].sum().reset_index(name='EmphasisDifferentiation')
emphasis_diff_df

,StockCode,Year,EmphasisDifferentiation
0,1,2006,0.000000
1,1,2007,0.002584
2,1,2008,0.496123
3,1,2009,0.252063
4,1,2010,0.169351
...,...,...,...
50915,920985,2020,0.000688
50916,920985,2021,0.000677
50917,920985,2022,0.000657
50918,920985,2023,0.000657


In [41]:
emphasis_diff_df.describe()

,StockCode,Year,EmphasisDifferentiation
count,50920.000000,50920.000000,50920.000000
mean,372755.376866,2015.000000,0.047047
std,280554.038736,5.477279,0.068023
min,1.000000,2006.000000,0.000000
25%,2562.500000,2010.000000,0.000866
50%,600069.000000,2015.000000,0.003364
75%,601515.750000,2020.000000,0.111010
max,920985.000000,2024.000000,0.993703


## 结果整合与输出

In [42]:
final_results = scope_conformity_df.merge(emphasis_diff_df, on=['StockCode', 'Year'], how='inner')
print(final_results.describe())
print("\n--- 最终计算结果 (Scope Conformity & Emphasis Differentiation) ---")
print(final_results.head(10))

           StockCode          Year  ScopeConformity  EmphasisDifferentiation
count   50920.000000  50920.000000     50920.000000             50920.000000
mean   372755.376866   2015.000000         0.171849                 0.047047
std    280554.038736      5.477279         0.280414                 0.068023
min         1.000000   2006.000000         0.000000                 0.000000
25%      2562.500000   2010.000000         0.000000                 0.000866
50%    600069.000000   2015.000000         0.000000                 0.003364
75%    601515.750000   2020.000000         0.332247                 0.111010
max    920985.000000   2024.000000         1.000000                 0.993703

--- 最终计算结果 (Scope Conformity & Emphasis Differentiation) ---
   StockCode  Year  ScopeConformity  EmphasisDifferentiation
0          1  2006         0.000000                 0.000000
1          1  2007         0.000000                 0.002584
2          1  2008         0.499999                 0.496123
3

## Z-score标准化

In [43]:
# 假设 final_results 是您的最终结果 DataFrame，包含 'ScopeConformity' 和 'EmphasisDifferentiation'

# 对 Scope Conformity 进行标准化
mean_scope = final_results['ScopeConformity'].mean()
std_scope = final_results['ScopeConformity'].std()
final_results['ScopeConformity_Z'] = (final_results['ScopeConformity'] - mean_scope) / std_scope

# 对 Emphasis Differentiation 进行标准化
mean_emphasis = final_results['EmphasisDifferentiation'].mean()
std_emphasis = final_results['EmphasisDifferentiation'].std()
final_results['EmphasisDifferentiation_Z'] = (final_results['EmphasisDifferentiation'] - mean_emphasis) / std_emphasis
print(final_results[['ScopeConformity_Z', 'EmphasisDifferentiation_Z']].describe())
final_results[['StockCode', 'Year', 'ScopeConformity_Z', 'EmphasisDifferentiation_Z']].head()


       ScopeConformity_Z  EmphasisDifferentiation_Z
count       5.092000e+04               5.092000e+04
mean       -1.250287e-16              -1.786125e-16
std         1.000000e+00               1.000000e+00
min        -6.128406e-01              -6.916329e-01
25%        -6.128406e-01              -6.788992e-01
50%        -6.128406e-01              -6.421790e-01
75%         5.720015e-01               9.403206e-01
max         2.953309e+00               1.391678e+01


,StockCode,Year,ScopeConformity_Z,EmphasisDifferentiation_Z
0,1,2006,-0.612841,-0.691633
1,1,2007,-0.612841,-0.653646
2,1,2008,1.170230,6.601860
3,1,2009,0.278696,3.013938
4,1,2010,1.795703,1.797986


In [44]:
final_results.to_csv('final_results.csv',encoding='utf-8-sig',index=False)